# NOAA-20 NUCAPS 2023-09-10 11:46 UTC

In [4]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
from six import StringIO
import pandas as pd

'''
NOAA-20 NUCAPS Profile at 2023-09-10 11:46 UTC
Retrieval Location: Lat: 52.8N Lon: 0.9E
'''

data_txt = '''
1000    24    13
950     26     9
920     25     8
850     18     9
780     13     9
700      7     4
670      5     1
620      1    -4
570     -5   -12
500    -13   -21
475    -15   -24
430    -20   -28
400    -24   -27
350    -30   -33
300    -37   -40
250    -46   -48
200    -58   -58
150    -62   -71
135    -62   -74
115    -62   -77
100    -61   -79
'''

nucaps_prof = StringIO(data_txt)
pressure_mb,temperature_c,dewpoint_c = np.loadtxt(nucaps_prof, usecols=range(0, 3), unpack=True)

prof_hgt = '''
0.0
0.5
0.8
1.5
2.2
3.0
3.3
4.0
4.5
5.5
6.0
6.7
7.2
8.0
9.0
10.4
11.8
13.5
14.3
15.3
16.25
'''
height_km = StringIO(prof_hgt)
height_km = np.loadtxt(height_km, usecols=range(0, 0), unpack=True)
height_m = height_km * 1000
print(height_m,len(height_m))

RH = 100*(np.exp((17.625*dewpoint_c)/(243.04+dewpoint_c))/np.exp((17.625*temperature_c)/(243.04+temperature_c)))
print("RH: ", RH)
L_vapor = 2500
thetae = (273.15 + temperature_c)*((1000/pressure_mb)**0.286)+(3 * (RH * (3.884266 * 10**
         ((7.5 * temperature_c)/(237.7 + temperature_c)))/100))
wetbulb = temperature_c*(np.arctan(0.151977*((RH+8.313659)**0.5))) + (np.arctan(temperature_c+RH)) - (np.arctan(RH-1.676331))+((0.00391838*(RH**1.5))*(np.arctan(0.023101*RH))) - 4.686035
WBD = temperature_c - wetbulb
satmixrat = (6.11*(10**((7.5*temperature_c)/(237.7+temperature_c))))/1000
print("Sat_mix_ratio = ",satmixrat)
mixrat = (6.11*(10**((7.5*dewpoint_c)/(237.7+dewpoint_c))))/1000
print("Mix_ratio = ",mixrat)
thetaw = thetae - (L_vapor*mixrat)
temperature_k = temperature_c + 273.15
tempvirt = temperature_k*(1+(0.61*(mixrat)))
tempvirt_c = tempvirt - 273.15
print("Virtual Temperature (C) = ",tempvirt_c)
tempvirt_exc = tempvirt_c - temperature_c

idx_pup = np.where(pressure_mb == 700.00)
idx_plo = np.where(pressure_mb == 950.00)
print("PUP idx = ", idx_pup)
print("PLO idx = ", idx_plo)
Z_upper = height_km[idx_pup]
Z_lower = height_km[idx_plo]
T_sfc = temperature_k[idx_plo]
T_top = temperature_k[idx_pup]
Te_sfc = thetae[idx_plo]
Te_top = thetae[idx_pup]
Tv_sfc = tempvirt_c[idx_plo]
Tv_top = tempvirt_c[idx_pup]
wetbulb_sfc = wetbulb[idx_plo]
wetbulb_top = wetbulb[idx_pup]
thetaw_sfc = thetaw[idx_pup]
delta_z = 2500
delta_zkm = 2.5
gamma = (T_sfc - T_top)/(delta_zkm)
gamma_Te = (Te_sfc - Te_top)/(Z_lower - Z_upper)
gamma_Tv = (Tv_sfc - Tv_top)/(delta_zkm)
gamma_wb = (wetbulb_sfc - wetbulb_top)/(delta_zkm)

print("")
print("Gamma = ", gamma)
if gamma > 4.8 and gamma <= 9.8:
    print("Conditional Instability") 
elif gamma > 9.8:
    print("Absolute Instability")
print("Theta-e Gamma = ", gamma_Te)
if gamma_Te < 0:
    print("Theta-e lapse rate is negative: potential instability")    
print("Tv Gamma = ", gamma_Tv)
print("Wetbulb Gamma = ", gamma_wb)
if gamma_wb > 4.8:
    print("Potential Instability")    

P_level_upper = pressure_mb[idx_pup]
P_level_lower = pressure_mb[idx_plo]
Z_upper = height_km[idx_pup]
Z_lower = height_km[idx_plo]
T_upper = temperature_c[idx_pup]
T_lower = temperature_c[idx_plo]
Tv_upper = tempvirt_c[idx_pup]
Tv_lower = tempvirt_c[idx_plo]
TD_upper = dewpoint_c[idx_pup]
TD_lower = dewpoint_c[idx_plo]
CAPE = 0
 
def MWPI(P_level_upper, P_level_lower, Z_upper, Z_lower, T_upper, T_lower, TD_upper, TD_lower, CAPE):
        gamma = (T_lower - T_upper)/(Z_upper - Z_lower)
        DD_upper = T_upper - TD_upper
        DD_lower = T_lower - TD_lower
        DDD = DD_lower - DD_upper
        MWPI = (CAPE/1000) + (gamma/5) + (DDD/5)
        WGP = (0.35435365777 * (MWPI**2)) + (1.2959855*MWPI) + 33.8176788
        return MWPI, WGP    
MWPI, WGP  = MWPI(P_level_upper, P_level_lower, Z_upper, Z_lower, T_upper, T_lower, TD_upper, TD_lower, CAPE)    
print("")
print("MWPI = ", MWPI)
print("MWPI WGP (kt) = ", WGP)

def MWPI_Tv(P_level_upper, P_level_lower, Z_upper, Z_lower, Tv_upper, Tv_lower, TD_upper, TD_lower, CAPE):
        gamma = (Tv_lower - Tv_upper)/(Z_upper - Z_lower)
        DD_upper = Tv_upper - TD_upper
        DD_lower = Tv_lower - TD_lower
        DDD = DD_lower - DD_upper
        MWPI_Tv = (CAPE/1000) + (gamma/5) + (DDD/5)
        WGP_Tv = (0.35435365777 * (MWPI_Tv**2)) + (1.2959855*MWPI_Tv) + 33.8176788
        return MWPI_Tv, WGP_Tv
MWPI_Tv, WGP_Tv  = MWPI_Tv(P_level_upper, P_level_lower, Z_upper, Z_lower, Tv_upper, Tv_lower, TD_upper, TD_lower, CAPE)    
print("")
print("MWPI from Virtual Temperature:")
print("MWPI = ", MWPI_Tv)
print("MWPI WGP (kt) = ", WGP_Tv)

mydata=dict(zip(('hght','pres','temp','dwpt','thtae','thetaw','wetbulb','depression','tempvirt','tempvirt_exc'),(height_m,pressure_mb,temperature_c,dewpoint_c,thetae,thetaw,wetbulb,WBD,tempvirt_c,tempvirt_exc)))
print(mydata)
df_ret = pd.DataFrame.from_dict(mydata)
print(df_ret)
wdir = []
wspd = []
wval = -9999.00
data_len = len(df_ret)
print("DF Length = ", data_len)
for i in range(data_len):
    wdir.append(wval)
    wspd.append(wval)
print("wdir, wspd: ", wdir, wspd)
data_sharppy=dict(zip(('pres','hght','temp','dwpt'),(pressure_mb, height_m, temperature_c, dewpoint_c)))
print(data_sharppy)
df_sharppy = pd.DataFrame.from_dict(data_sharppy)
print(df_sharppy)
df_sharppy = df_sharppy[['pres', 'hght', 'temp', 'dwpt']]
print(df_sharppy)
df_sharppy_wind =df_sharppy.assign(wdir = wdir, wspd = wspd)
print(df_sharppy_wind)
print("")
print("Sharppy formatted retrieval: ")
print("")
print(df_sharppy_wind.to_string(header=False, index=False, formatters={"pres": "  {:.2f},  ".format, "hght": "{:.2f},  ".format,
                                                                       "temp": "{:.2f},  ".format, "dwpt": "{:.2f},  ".format,
                                                                       "wdir": "{:.2f},  ".format, "wspd": "{:.2f} ".format}))
print("")

(array([    0.,   500.,   800.,  1500.,  2200.,  3000.,  3300.,  4000.,
        4500.,  5500.,  6000.,  6700.,  7200.,  8000.,  9000., 10400.,
       11800., 13500., 14300., 15300., 16250.]), 21)
('RH: ', array([ 50.20012605,  34.16686278,  33.88443914,  55.6567625 ,
        76.68106955,  81.21727894,  75.34718121,  69.27047022,
        57.97171242,  51.12446935,  46.21720541,  48.94427049,
        76.21872929,  75.03712986,  73.55171267,  80.01748538,
       100.        ,  29.00257479,  18.64353903,  11.79465894,
         7.56196573]))
('Sat_mix_ratio = ', array([2.97749938e-02, 3.35365118e-02, 3.16069887e-02, 2.06064731e-02,
       1.49607119e-02, 1.00135861e-02, 8.72073226e-03, 6.56842738e-03,
       4.21589235e-03, 2.24972445e-03, 1.90931380e-03, 1.25031552e-03,
       8.78494238e-04, 5.04368551e-04, 2.53151510e-04, 9.69067881e-05,
       2.31920982e-05, 1.37874700e-05, 1.37874700e-05, 1.37874700e-05,
       1.57364615e-05]))
('Mix_ratio = ', array([1.49607119e-02, 1.14723631e-02, 

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from skewt import SkewT
import re
from six import StringIO
import pandas as pd

'''
03354 Nottingham RAOB Profile at 1200 UTC 10 September 2023 
'''

data_txt = '''
 1002.0    117   23.4   18.5     74  13.56     70      6  296.4  335.7  298.8
 1000.0    135   22.8   18.4     76  13.50     70      4  295.9  335.0  298.4
  998.0    152   22.4   18.1     77  13.27     73      4  295.7  334.1  298.1
  968.0    418   20.0   17.4     85  13.08    118      9  295.9  333.7  298.2
  964.0    454   20.2   17.1     82  12.89    124      9  296.4  333.8  298.7
  960.0    490   21.4   15.4     69  11.59    130     10  298.0  331.9  300.1
  954.0    544   21.8   14.8     64  11.21    131     10  298.9  331.9  301.0
  934.0    728   20.9   14.7     68  11.34    135     12  299.8  333.3  301.9
  928.0    784   20.6   14.6     68  11.38    138     12  300.1  333.7  302.1
  925.0    812   20.4   14.4     68  11.27    140     12  300.2  333.5  302.2
  923.0    831   20.2   14.2     68  11.15    141     12  300.1  333.1  302.1
  906.0    991   19.0   14.2     74  11.36    148     13  300.5  334.1  302.6
  896.0   1087   18.4   12.4     68  10.19    152     13  300.8  331.1  302.7
  888.0   1163   17.8   12.4     71  10.27    155     13  300.9  331.5  302.8
  859.0   1447   15.4   12.3     82  10.58    160     14  301.3  332.8  303.2
  850.0   1537   14.6   12.3     86  10.68    160     15  301.4  333.2  303.4
  840.0   1637   13.8   12.1     89  10.67    166     16  301.6  333.3  303.5
  835.0   1688   13.6   11.4     87  10.24    169     17  301.9  332.4  303.8
  834.0   1698   13.6   10.5     81   9.62    170     17  302.0  330.8  303.8
  832.0   1718   13.6    8.6     72   8.49    171     17  302.2  327.7  303.8
  827.0   1769   13.5    8.0     69   8.20    175     17  302.6  327.3  304.1
  822.0   1820   13.4    7.4     67   7.92    183     16  303.1  327.0  304.5
  818.0   1861   13.1    6.8     65   7.60    190     16  303.2  326.2  304.6
  811.0   1933   12.6    5.6     62   7.08    191     15  303.4  324.9  304.7
  806.0   1985   12.2    4.2     58   6.45    192     14  303.5  323.2  304.7
  800.0   2048   11.8    2.8     54   5.89    193     14  303.7  321.8  304.8
  788.0   2174   10.8    3.3     60   6.20    195     12  303.9  322.9  305.1
  777.0   2292    9.8    3.8     66   6.51    199     13  304.1  324.0  305.3
  773.0   2334    9.6    1.6     57   5.59    200     14  304.3  321.6  305.4
  769.0   2378    9.4    1.4     57   5.54    202     14  304.6  321.6  305.6
  766.0   2410    9.0    4.0     71   6.70    203     14  304.5  325.0  305.7
  761.0   2464    8.8    4.7     75   7.08    205     15  304.8  326.5  306.1
  760.0   2475    8.7    4.6     76   7.06    205     15  304.8  326.4  306.1
  753.0   2552    8.0    4.2     77   6.91    205     17  304.9  326.0  306.2
  747.0   2618    7.8    3.2     73   6.51    205     19  305.3  325.3  306.5
  743.0   2662    7.6    2.6     71   6.25    203     20  305.6  324.9  306.8
  727.0   2841    6.4    2.2     75   6.23    195     25  306.2  325.4  307.3
  725.0   2864    6.2    2.2     76   6.22    194     25  306.2  325.5  307.4
  723.0   2887    6.0    1.2     71   5.81    194     25  306.3  324.2  307.3
  722.0   2898    6.2   -1.8     57   4.67    193     25  306.6  321.2  307.5
  720.0   2921    6.0   -4.0     49   3.97    192     26  306.6  319.2  307.4
  706.0   3081    4.8   -2.2     60   4.63    187     27  307.0  321.6  307.9
  700.0   3151    4.2   -1.8     65   4.81    185     27  307.1  322.2  308.0
  697.0   3186    3.9   -2.5     63   4.59    185     27  307.2  321.6  308.1
  693.0   3233    3.6   -3.4     60   4.31    185     27  307.3  320.9  308.1
  691.0   3256    3.6   -1.4     70   5.02    186     28  307.6  323.3  308.5
  682.0   3362    3.0    0.2     82   5.73    186     29  308.1  325.9  309.1
  672.0   3482    1.8    0.1     88   5.77    187     30  308.0  326.0  309.1
  661.0   3615    0.8   -0.4     92   5.65    188     31  308.4  326.0  309.4
  658.0   3651    0.8   -1.0     88   5.44    188     32  308.8  325.8  309.8
  639.0   3886   -0.8   -1.4     95   5.41    190     34  309.5  326.6  310.6
  633.0   3961   -1.3   -1.6     98   5.41    190     34  309.8  326.8  310.8
  614.0   4203   -2.7   -2.7    100   5.14    190     33  310.9  327.2  311.9
  583.0   4611   -5.2   -5.2    100   4.49    190     32  312.6  327.1  313.5
  568.0   4816   -6.4   -6.4    100   4.19    200     31  313.5  327.1  314.3
  567.0   4830   -6.5   -6.5    100   4.17    200     31  313.6  327.1  314.4
  564.0   4872   -6.9   -7.8     93   3.79    201     31  313.6  325.9  314.3
  545.0   5138   -9.1   -9.2     99   3.51    204     33  314.1  325.6  314.7
  536.0   5266   -9.8  -10.7     93   3.17    205     34  314.7  325.2  315.3
  532.0   5324  -10.1  -11.4     90   3.02    206     34  315.0  325.0  315.6
  530.0   5353  -10.1  -11.3     91   3.06    207     33  315.4  325.5  315.9
  526.0   5412  -10.5  -13.1     81   2.66    208     33  315.6  324.5  316.1
  523.0   5456  -10.7  -13.3     81   2.64    209     33  315.9  324.7  316.4
  521.0   5485  -10.7  -14.5     74   2.40    209     32  316.2  324.3  316.6
  519.0   5515  -10.9  -14.0     78   2.51    210     32  316.3  324.7  316.8
  518.0   5529  -11.0  -14.2     77   2.48    210     32  316.4  324.7  316.9
  514.0   5589  -11.3  -14.8     75   2.37    208     32  316.7  324.7  317.1
  500.0   5800  -12.5  -13.8     90   2.65    200     31  317.7  326.7  318.2
  499.0   5815  -12.5  -13.4     93   2.74    200     31  317.9  327.1  318.4
  491.0   5939  -13.3  -14.4     91   2.57    198     30  318.4  327.1  318.9
  488.0   5985  -13.1  -16.0     79   2.26    197     30  319.2  326.9  319.6
  486.0   6017  -13.3  -16.3     78   2.21    197     30  319.3  326.9  319.8
  485.0   6032  -13.3  -16.3     78   2.22    196     30  319.5  327.1  320.0
  461.0   6415  -16.1  -18.0     86   2.02    190     28  320.6  327.6  321.0
  450.0   6598  -17.5  -18.8     90   1.94    194     28  321.2  327.9  321.5
  447.0   6648  -17.8  -19.4     87   1.85    195     28  321.4  327.8  321.7
  443.0   6715  -18.3  -20.3     84   1.73    194     28  321.6  327.6  321.9
  442.0   6732  -18.5  -21.2     79   1.60    194     28  321.6  327.2  321.9
  438.0   6800  -19.1  -21.0     85   1.65    193     27  321.6  327.4  321.9
  436.0   6834  -19.3  -21.3     84   1.61    193     27  321.8  327.5  322.1
  434.0   6868  -19.5  -23.3     72   1.36    193     27  322.0  326.8  322.2
  433.0   6885  -19.5  -22.3     78   1.49    192     27  322.2  327.4  322.5
  431.0   6919  -19.7  -22.4     79   1.48    192     27  322.4  327.6  322.6
  421.0   7093  -20.4  -23.9     74   1.33    190     26  323.6  328.4  323.9
  420.0   7111  -20.5  -24.0     74   1.32    190     26  323.7  328.4  324.0
  404.0   7397  -22.7  -25.3     79   1.22    190     28  324.5  328.9  324.7
  400.0   7470  -23.3  -26.4     76   1.11    190     29  324.6  328.7  324.8
  396.0   7544  -23.9  -27.1     75   1.05    189     29  324.8  328.6  325.0
  389.0   7674  -24.7  -29.6     64   0.85    189     30  325.4  328.5  325.6
  383.0   7787  -25.5  -30.2     65   0.82    188     31  325.8  328.8  325.9
  381.0   7825  -25.9  -29.9     69   0.84    187     31  325.7  328.9  325.9
  379.0   7863  -26.1  -31.0     63   0.76    187     31  326.0  328.8  326.1
  373.0   7978  -26.9  -31.9     63   0.71    186     32  326.4  329.1  326.5
  371.0   8017  -27.1  -32.1     62   0.70    186     32  326.6  329.3  326.8
  369.0   8056  -27.5  -32.5     62   0.68    186     32  326.6  329.2  326.7
  366.0   8115  -27.7  -35.7     46   0.50    185     33  327.1  329.0  327.2
  363.0   8174  -28.1  -37.1     42   0.44    185     33  327.3  329.0  327.4
  361.0   8214  -28.5  -37.5     42   0.42    187     33  327.3  329.0  327.4
  344.0   8555  -31.5  -39.0     48   0.38    200     34  327.8  329.3  327.9
  329.0   8871  -34.3  -40.3     54   0.35    200     36  328.1  329.5  328.2
  314.0   9196  -37.3  -40.2     74   0.37    200     38  328.4  329.8  328.4
  313.0   9218  -37.3  -43.3     54   0.27    200     38  328.7  329.7  328.7
  310.0   9285  -37.5  -45.5     43   0.21    200     39  329.3  330.2  329.3
  309.0   9307  -37.7  -45.7     43   0.21    200     39  329.3  330.2  329.4
  304.0   9419  -38.9  -45.9     47   0.21    200     39  329.2  330.0  329.2
  301.0   9487  -39.5  -47.5     42   0.17    200     40  329.3  330.0  329.3
  300.0   9510  -39.7  -47.7     42   0.17    200     40  329.3  330.0  329.3
  294.0   9648  -40.7  -46.7     52   0.20    200     42  329.8  330.6  329.8
  292.0   9694  -41.1  -48.1     47   0.17    200     43  329.9  330.6  329.9
  286.0   9835  -42.5  -48.5     52   0.16    200     45  329.8  330.5  329.9
  285.0   9859  -42.6  -49.1     49   0.15    200     45  330.0  330.6  330.0
  282.0   9930  -42.9  -50.9     41   0.13    200     45  330.6  331.1  330.6
  279.0  10002  -43.5  -50.5     46   0.13    201     44  330.7  331.3  330.8
  277.0  10050  -43.7  -51.7     41   0.12    201     44  331.1  331.6  331.1
  272.0  10173  -45.1  -51.1     51   0.13    202     43  330.8  331.4  330.8
  263.0  10396  -46.9  -51.5     59   0.13    203     42  331.4  331.9  331.4
  261.0  10447  -47.5  -53.5     50   0.10    204     41  331.2  331.6  331.2
  258.0  10523  -48.1  -54.1     50   0.09    204     41  331.4  331.8  331.4
  254.0  10626  -49.0  -55.0     49   0.09    205     40  331.6  331.9  331.6
  251.0  10704  -49.7  -55.7     49   0.08    201     41  331.7  332.0  331.7
  250.0  10730  -49.9  -55.9     49   0.08    200     41  331.8  332.1  331.8
  244.0  10889  -50.9  -57.9     43   0.06    198     41  332.6  332.8  332.6
  237.0  11078  -52.9  -58.9     48   0.06    196     42  332.3  332.6  332.3
  234.0  11161  -53.5  -59.5     48   0.05    195     42  332.6  332.9  332.6
  227.0  11356  -54.9  -60.9     47   0.05    192     43  333.4  333.6  333.4
  225.0  11413  -55.3  -62.3     41   0.04    191     43  333.6  333.8  333.6
  221.0  11528  -55.8  -61.5     49   0.04    190     43  334.5  334.7  334.5
  219.0  11586  -56.1  -61.1     53   0.05    191     44  335.0  335.2  335.0
  211.0  11822  -57.9  -63.9     46   0.03    195     49  335.7  335.9  335.8
  207.0  11943  -58.5  -63.5     52   0.04    197     52  336.6  336.8  336.6
  205.0  12004  -58.7  -63.7     52   0.04    198     53  337.3  337.4  337.3
  203.0  12066  -58.7  -64.7     46   0.03    199     54  338.2  338.4  338.2
  200.0  12160  -59.1  -64.0     53   0.03    200     56  339.0  339.2  339.0
  199.0  12191  -59.3  -63.8     56   0.04    200     57  339.2  339.4  339.2
  198.0  12223  -59.5  -63.8     57   0.04    200     57  339.4  339.5  339.4
  190.0  12478  -61.1  -64.1     67   0.04    200     53  340.8  341.0  340.8
  189.0  12511  -61.3  -64.4     66   0.04    200     53  341.0  341.2  341.0
  185.0  12643  -62.3  -64.7     73   0.03    200     51  341.5  341.6  341.5
  184.0  12676  -61.9  -65.3     64   0.03    201     50  342.6  342.8  342.7
  182.0  12744  -60.5  -69.5     30   0.02    202     49  346.0  346.1  346.0
  181.0  12778  -60.1  -71.1     22   0.01    203     48  347.2  347.3  347.2
  179.0  12847  -59.5  -73.0     16   0.01    205     46  349.3  349.3  349.3
  177.0  12917  -58.9  -74.9     11   0.01    202     46  351.4  351.4  351.4
  176.0  12952  -58.7  -76.7      8   0.01    201     46  352.3  352.3  352.3
  170.0  13169  -59.9  -75.9     11   0.01    192     45  353.8  353.8  353.8
  169.0  13205  -59.8  -76.2     10   0.01    190     45  354.6  354.6  354.6
  163.0  13431  -59.2  -77.9      7   0.01    195     47  359.2  359.2  359.2
  157.0  13665  -58.6  -79.7      5   0.00    215     39  364.1  364.1  364.1
  150.0  13950  -57.9  -81.9      3   0.00    210     33  370.1  370.1  370.1
  149.0  13992  -57.5  -81.5      3   0.00    210     34  371.5  371.5  371.5
  147.0  14078  -56.7  -80.7      3   0.00    214     31  374.3  374.4  374.3
  144.0  14208  -56.9  -80.8      3   0.00    220     26  376.3  376.3  376.3
  140.0  14387  -57.0  -81.0      3   0.00    205     27  379.0  379.0  379.0
  139.0  14432  -57.1  -81.1      3   0.00    207     27  379.7  379.7  379.7
  136.0  14571  -55.5  -81.5      3   0.00    213     26  384.9  384.9  384.9
  133.0  14712  -56.1  -81.6      3   0.00    220     26  386.2  386.2  386.2
  128.0  14955  -57.2  -81.7      3   0.00    205     21  388.5  388.5  388.5
  126.0  15055  -57.7  -81.7      3   0.00    200     23  389.4  389.4  389.4
  122.0  15259  -56.9  -81.9      3   0.00    230     26  394.4  394.5  394.4
  116.0  15576  -57.9  -82.5      3   0.00    265     15  398.3  398.4  398.3
  114.0  15686  -58.2  -82.7      3   0.00    270     10  399.7  399.7  399.7
  111.0  15854  -58.8  -83.0      3   0.00    240      5  401.8  401.8  401.8
  109.0  15968  -59.1  -83.2      3   0.00    190     10  403.2  403.2  403.2
  108.0  16026  -59.3  -83.3      3   0.00    197     12  403.9  403.9  403.9
  107.0  16085  -58.7  -83.7      3   0.00    205     15  406.1  406.1  406.1
  106.0  16144  -58.1  -84.1      2   0.00    206     16  408.3  408.4  408.3
  100.0  16510  -58.9  -84.9      2   0.00    215     20  413.6  413.7  413.6
   98.0  16637  -58.8  -85.0      2   0.00    200     14  416.2  416.2  416.2
   92.0  17036  -58.6  -85.3      2   0.00    240     17  424.1  424.1  424.1
   90.0  17174  -58.6  -85.4      2   0.00    245     15  426.9  426.9  426.9
   88.0  17316  -58.5  -85.5      2   0.00    210     11  429.8  429.8  429.8
   86.0  17461  -58.5  -85.6      2   0.00    210     10  432.8  432.8  432.8
   84.0  17609  -58.4  -85.7      2   0.00    200     12  435.8  435.9  435.8
   82.0  17761  -58.3  -85.8      2   0.00    215     13  439.0  439.0  439.0
   76.6  18190  -58.1  -86.1      2   0.00    218     17  448.1  448.1  448.1
   74.0  18408  -57.8  -86.2      2   0.00    220     19  453.1  453.2  453.1
   71.0  18670  -57.4  -86.3      1   0.00    245     10  459.3  459.3  459.3
   70.0  18760  -57.3  -86.3      1   0.00    240      5  461.4  461.5  461.4
   67.0  19038  -56.7  -86.6      1   0.00    185     10  468.6  468.6  468.6
   64.0  19328  -56.1  -86.8      1   0.00    220     11  476.1  476.1  476.1
   63.2  19408  -55.9  -86.9      1   0.00    216     12  478.2  478.2  478.2
   62.0  19529  -56.1  -87.0      1   0.00    210     13  480.4  480.5  480.4
   60.0  19736  -56.4  -87.1      1   0.00    235     15  484.3  484.4  484.3
   59.0  19843  -56.5  -87.2      1   0.00    245     11  486.3  486.3  486.3
   55.0  20287  -57.1  -87.5      1   0.00    210      4  494.7  494.8  494.7
   51.0  20765  -57.8  -87.9      1   0.00    175     10  503.9  504.0  503.9
   50.5  20827  -57.9  -87.9      1   0.00    182     11  505.2  505.2  505.2
   50.0  20890  -57.7  -88.7      1   0.00    190     13  507.1  507.1  507.1
   49.0  21018  -57.5  -88.7      1   0.00    210     14  510.4  510.4  510.4
   48.0  21149  -57.4  -88.7      1   0.00    225     10  513.9  513.9  513.9
   39.7  22351  -55.7  -88.7      1   0.00    234     10  546.6  546.7  546.6
   35.2  23114  -57.1  -89.1      1   0.00    240     10  562.1  562.1  562.1
   35.0  23150  -57.0  -89.1      1   0.00    240     10  563.4  563.4  563.4
   34.0  23334  -56.2  -89.1      1   0.00    270     10  570.0  570.0  570.0
   32.5  23621  -55.1  -89.1      1   0.00    254      7  580.4  580.5  580.4
   30.0  24130  -56.5  -89.5      1   0.00    225      3  590.0  590.1  590.0
   29.0  24346  -56.4  -89.7      1   0.00      0      0  596.1  596.1  596.1
   26.8  24848  -56.1  -90.1      1   0.00     98      7  610.5  610.5  610.5
   26.0  25043  -55.2  -89.8      1   0.00    135     10  618.3  618.3  618.3
   25.0  25295  -54.1  -89.4      1   0.01    155     11  628.5  628.6  628.5
   24.0  25558  -52.9  -89.0      1   0.01    195     15  639.4  639.5  639.4
   23.0  25831  -51.6  -88.5      0   0.01    315     10  650.9  650.9  650.9
   22.9  25860  -51.5  -88.5      0   0.01    310      9  652.1  652.1  652.1
   22.0  26120  -51.4  -88.9      0   0.01    260      2  659.9  660.0  659.9
   21.2  26361  -51.3  -89.3      0   0.01    228      8  667.2  667.3  667.2
   21.0  26423  -51.4  -89.2      0   0.01    220     10  668.7  668.8  668.7
   20.0  26740  -51.9  -88.9      0   0.01    290      9  676.6  676.6  676.6
   19.6  26871  -52.3  -89.3      0   0.01    253      9  679.2  679.3  679.2
   19.0  27073  -50.6  -88.0      0   0.01    195     10  690.6  690.7  690.6
   18.3  27318  -48.5  -86.5      0   0.01    240     10  704.6  704.8  704.6
   18.0  27427  -48.5  -86.5      0   0.01    260     10  707.8  708.0  707.8
   17.0  27802  -48.7  -86.7      0   0.01    290     12  719.1  719.2  719.1
   16.0  28200  -48.8  -86.8      0   0.01    305     10  731.2  731.4  731.2
   15.0  28624  -48.9  -86.9      0   0.01    225     10  744.4  744.5  744.4
   14.0  29077  -49.1  -87.1      0   0.01    230      9  758.7  758.8  758.7
   13.9  29124  -49.1  -87.1      0   0.01                760.2  760.4  760.2  
'''
raobprof = StringIO(data_txt)
pressure_mb,height,temperature_c,dewpoint_c,RH_pct,mixrat,wdir,wspd = np.loadtxt(raobprof, usecols=range(0, 8), unpack=True)

press = pressure_mb.tolist()
print("Pressure : ", press, len(press))
height_m = height.tolist()
print("Height : ", height_m, len(height_m))
 
data_sharppy=dict(zip(('pres','hght','temp','dwpt','wdir','wspd'),(pressure_mb, height, temperature_c, dewpoint_c, wdir, wspd)))
print(data_sharppy)
df_sharppy = pd.DataFrame.from_dict(data_sharppy)
print(df_sharppy)
df_sharppy = df_sharppy[['pres', 'hght', 'temp', 'dwpt', 'wdir', 'wspd']]
print(df_sharppy)
print("")
print("Sharppy formatted retrieval: ")
print("")
print(df_sharppy.to_string(header=False, index=False, formatters={"pres": "  {:.2f},  ".format, "hght": "{:.2f},  ".format,
                                                                       "temp": "{:.2f},  ".format, "dwpt": "{:.2f},  ".format,
                                                                       "wdir": "{:.2f},  ".format, "wspd": "{:.2f} ".format}))
print("")

('Pressure : ', [1002.0, 1000.0, 998.0, 968.0, 964.0, 960.0, 954.0, 934.0, 928.0, 925.0, 923.0, 906.0, 896.0, 888.0, 859.0, 850.0, 840.0, 835.0, 834.0, 832.0, 827.0, 822.0, 818.0, 811.0, 806.0, 800.0, 788.0, 777.0, 773.0, 769.0, 766.0, 761.0, 760.0, 753.0, 747.0, 743.0, 727.0, 725.0, 723.0, 722.0, 720.0, 706.0, 700.0, 697.0, 693.0, 691.0, 682.0, 672.0, 661.0, 658.0, 639.0, 633.0, 614.0, 583.0, 568.0, 567.0, 564.0, 545.0, 536.0, 532.0, 530.0, 526.0, 523.0, 521.0, 519.0, 518.0, 514.0, 500.0, 499.0, 491.0, 488.0, 486.0, 485.0, 461.0, 450.0, 447.0, 443.0, 442.0, 438.0, 436.0, 434.0, 433.0, 431.0, 421.0, 420.0, 404.0, 400.0, 396.0, 389.0, 383.0, 381.0, 379.0, 373.0, 371.0, 369.0, 366.0, 363.0, 361.0, 344.0, 329.0, 314.0, 313.0, 310.0, 309.0, 304.0, 301.0, 300.0, 294.0, 292.0, 286.0, 285.0, 282.0, 279.0, 277.0, 272.0, 263.0, 261.0, 258.0, 254.0, 251.0, 250.0, 244.0, 237.0, 234.0, 227.0, 225.0, 221.0, 219.0, 211.0, 207.0, 205.0, 203.0, 200.0, 199.0, 198.0, 190.0, 189.0, 185.0, 184.0, 182.0, 1